In [17]:
import rosbag
import pickle
import rospy
import datetime
import msgpack
import gc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob2, re
from collections import Counter
import networkx as nx
import community as comm
import seaborn as sns
from joblib import Parallel, delayed 
%matplotlib inline

# Data filtering and conversion

In [2]:
def TopicFilter(bag, BASE_ROBOT_ID):

    topics = bag.get_type_and_topic_info().topics.keys()
    topics_excluing_list = []
    topics_valid_list = []

    # remove topics that appeared in argos
    regex = re.compile('^/epuck+_.+_.+/')
    argos_sim_topics = filter(regex.search, topics)
        
    # count the number of robots present in the rosbag file
    regex = re.compile('^/epuck+_.+/init_merkle_tree')
    number_of_robots_in_bag = len(filter(regex.search, topics))
    
    for id in xrange(BASE_ROBOT_ID,BASE_ROBOT_ID + number_of_robots_in_bag):
        topic_goal = "/epuck_" + str(id) + "/diff_drive_go_to_goal" + "/goal"
        topic_goal_achieved = "/epuck_" + str(id) + "/diff_drive_go_to_goal" + "/goal_achieved"
        topic_distance_to_goal = "/epuck_" + str(id) + "/diff_drive_go_to_goal" + "/distance_to_goal"
        topic_goto = "/epuck_" + str(id) + "/diff_drive_go_to_goal" + "/go_to"
        topic_results = "/epuck_" + str(id) + "/diff_drive_go_to_goal" + "/result"
        topic_cancel = "/epuck_" + str(id) + "/diff_drive_go_to_goal" + "/cancel"
        topic_status = "/epuck_" + str(id) + "/diff_drive_go_to_goal" + "/status"
        topic_cmd_vel = "/epuck_" + str(id) + "/cmd_vel"

        topics_excluing_list.extend([topic_goal, topic_distance_to_goal, topic_goto, 
                                     topic_goal_achieved, topic_results, topic_cancel, 
                                     topic_status, topic_cmd_vel])

    topics_excluing_list.extend(['/rosout', '/rosout_agg'])
    topics_excluing_list.extend(argos_sim_topics)

    for topic in topics:
        if topic not in topics_excluing_list:
            topics_valid_list.append(topic)
          
    return topics_valid_list

In [3]:
def ROSBagToPandasDataFrame(rosbag_filename, convert_to_csv=False, convert_to_pickle=False, convert_to_msgpack=False, convert_to_hdf5=False):

    bag = rosbag.Bag(rosbag_filename)
    topics_valid_list = TopicFilter(bag, 20)
    msgs = []

    # load the messages into the list
    for topic, msg, t in bag.read_messages(topics=topics_valid_list):
        msgs.append({'topic': topic, 'msg': msg, 't': t})
    
    # close bag
    bag.close()
    
    # robsag to pandas
    df = pd.DataFrame(msgs)
    
    # convert timestamp to pandas datatime format
    timestamps_converted = [str(i)[:13] for i in df['t']]
    df['t'] = pd.to_datetime(timestamps_converted, unit='ms')
    df=df.set_index('t')
    
    if (convert_to_csv):
        df.to_csv(rosbag_filename +'.csv')
    
    if (convert_to_pickle):
        df.to_pickle(rosbag_filename +'.pkl')
        
    if (convert_to_msgpack):
        # Write msgpack file
        with open(rosbag_filename +'.msgpack', "wb") as outfile:
            packed = msgpack.packb(df.to_msgpack())
            outfile.write(packed)
            
    if (convert_to_hdf5):
        df.to_hdf(rosbag_filename +'.hdf5', key='df', mode='w')
            
    # return dataframe
    return df

#  Finishing Time and Success Rate Functions

In [4]:
def MerkleTreeCompletionTimePerRobot(df, robot_topic):
    msgs_topic = df.loc[df['topic'] == robot_topic]['msg']
    msg_begin = msgs_topic.iloc[0]
    msg_end = None
    
    for msg in msgs_topic:
        if(msg.leafs[-1].completed == True):
            msg_end = msg
            break
     
    return msg_end.header.stamp.secs - msg_begin.header.stamp.secs, df.loc[df['msg'] == msg_end].index[0]

In [5]:
def FinishingTimeLastRobotToCompleteMerkleTree(INITIAL_ID, NUMBER_OF_ROBOTS, THRESHOLD, DF):
    robots_finishing_list = []

    for id in xrange(INITIAL_ID, INITIAL_ID+NUMBER_OF_ROBOTS):
        topic_name = '/epuck_' + str(id) + '/comm/merkle_completed'
        try:
            robot_finishing_time = MerkleTreeCompletionTimePerRobot(DF, topic_name)
            if (robot_finishing_time < THRESHOLD):
                robots_finishing_list.append(robot_finishing_time)
        except:
            pass
           
    return max(robots_finishing_list)

In [6]:
def FinishingTimeFirstRobotToCompleteMerkleTree(INITIAL_ID, NUMBER_OF_ROBOTS, THRESHOLD, DF):
    robots_finishing_list = []
    robots_finishing_timestamps = []

    for id in xrange(INITIAL_ID, INITIAL_ID+NUMBER_OF_ROBOTS):
        topic_name = '/epuck_' + str(id) + '/comm/merkle_completed'
        try:
            robot_finishing_time, robot_finishing_timestamp = MerkleTreeCompletionTimePerRobot(DF, topic_name)
            if (robot_finishing_time < THRESHOLD):
                robots_finishing_list.append(robot_finishing_time)
                robots_finishing_timestamps.append(robot_finishing_timestamp)
        except:
            pass
           
    return min(i for i in robots_finishing_list if i > 0), min(robots_finishing_timestamps)

In [7]:
def CompletionSuccess(THRESHOLD, FINISHING_TIMES):

    filtered_times = []

    for t in range(0,THRESHOLD):
        filtered_times.append(len([i for i in FINISHING_TIMES if i <= t]))
  
    return np.true_divide(np.array(filtered_times), len(FINISHING_TIMES))

# Communication Cost and Privacy Functions

In [8]:
# function to get unique values 
def unique(list1): 
    x = np.array(list1)
    return np.unique(x)

In [9]:
def sdi(data):
    from math import log as ln
    
    def p(n, N):
        if n is  0:
            return 0
        else:
            return (float(n)/N) * ln(float(n)/N)
            
    N = sum(data)
    return -sum(p(n, N) for n in data if n is not 0)

In [10]:
# get_size function
def get_size(obj, seen=None):
    """Recursively finds size of objects"""
    size = sys.getsizeof(obj)
    if seen is None:
        seen = set()
    obj_id = id(obj)
    if obj_id in seen:
        return 0
    # Important mark as seen *before* entering recursion to gracefully handle
    # self-referential objects
    seen.add(obj_id)
    if isinstance(obj, dict):
        size += sum([get_size(v, seen) for v in obj.values()])
        size += sum([get_size(k, seen) for k in obj.keys()])
    elif hasattr(obj, '__dict__'):
        size += get_size(obj.__dict__, seen)
    elif hasattr(obj, '__iter__') and not isinstance(obj, (str, bytes, bytearray)):
        size += sum([get_size(i, seen) for i in obj])
    return size

In [11]:
def ExtractCommunicationGraphEndOfExperiment(INITIAL_ID, NUMBER_OF_ROBOTS, NUMBER_OF_LEAVES, DF):
    G = nx.DiGraph()
    for robotid in xrange(INITIAL_ID, INITIAL_ID+NUMBER_OF_ROBOTS):
        G.add_node(robotid)
        msgs_topic = DF.loc[DF['topic'] == '/epuck_' + str(robotid) + '/comm/merkle_completed']['msg']
        from_robots = []
        for leaf in msgs_topic.iloc[-1].leafs:
            if leaf.received == True:
                from_robots.append(leaf.from_robot)         
        for other_robot in unique(from_robots):
            G.add_edge(other_robot, robotid, weight=Counter(from_robots)[other_robot])
    return G

In [12]:
def ExtractCommunicationGraphFirstRobotToCompleteMerkleTree(INITIAL_ID, NUMBER_OF_ROBOTS, NUMBER_OF_LEAVES, DF):
    G = nx.DiGraph()
    finishing_time, finishing_timestamp = FinishingTimeFirstRobotToCompleteMerkleTree(INITIAL_ID, NUMBER_OF_ROBOTS, 5100, DF)  
    for robotid in xrange(INITIAL_ID, INITIAL_ID+NUMBER_OF_ROBOTS):
        G.add_node(robotid)
        msgs_time_completed_merkle = DF.loc[(DF.index <= finishing_timestamp)]
        completion_msg = msgs_time_completed_merkle.loc[msgs_time_completed_merkle['topic'] == '/epuck_' + str(robotid) + '/comm/merkle_completed']['msg']
        last_completion_msg = completion_msg[-1]
        from_robots = []
        for leaf in last_completion_msg.leafs:
             if leaf.received == True:
                from_robots.append(leaf.from_robot)         
        for other_robot in unique(from_robots):
             G.add_edge(other_robot, robotid, weight=Counter(from_robots)[other_robot])
    return G

In [13]:
def CommunicationCost(G, NUMBER_OF_LEAVES, HASH_SIZE_IN_BYTES):  
    number_of_iterations = sum(nx.get_edge_attributes(G,'weight').values())    
    if (number_of_iterations!=0):
        return sum(nx.get_edge_attributes(G,'weight').values()) * (np.log2(NUMBER_OF_LEAVES) + 2) * HASH_SIZE_IN_BYTES

In [14]:
def PrivacyCoefficient(INITIAL_ID, NUMBER_OF_ROBOTS, NUMBER_OF_LEAVES, DF):
    by_all_robots_in_sim = []

    for robotid in xrange(INITIAL_ID, INITIAL_ID+NUMBER_OF_ROBOTS):
        msgs_topic = DF.loc[DF['topic'] == '/epuck_' + str(robotid) + '/comm/merkle_completed']['msg']
        by_this_robot = 0
        for leaf in msgs_topic.iloc[-1].leafs:
            if leaf.received == False and leaf.hash != "":
                by_this_robot =+ 1
        
        by_all_robots_in_sim.append(by_this_robot)

        if sum(by_all_robots_in_sim)==NUMBER_OF_LEAVES:
            return sdi(by_all_robots_in_sim)/np.log(NUMBER_OF_LEAVES)

# Multi-thread loop

In [15]:
path=r'/home/eddie/experiments_results/real_robots/maze_formation' 
ALL_FILES = glob2.glob(path + "/*/*/*.bag")
RUN_THRESHOLD = 5100
NUMBER_OF_ROBOTS_LIST = [16, 20, 24, 28]
INITIAL_ID = 20
NUMBER_OF_LEAVES = 16
HASH_SIZE_IN_BYTES = 32
LEAF_RANGE = range(16,17)
        
def RosbagDataProcessing(num_of_bots):   
    FINISHING_TIMES      = []
    COMPLETION_SUCCESS   = []
    COMMUNICATION_GRAPHS = []
    PRIVACY_COEFFICIENT  = []
    COMMUNICATION_COST   = []
    
    regex = re.compile(r'R' + str(num_of_bots))  
    FILTERED_FILES = filter(regex.search, ALL_FILES)

    for _file in FILTERED_FILES:
        DF = ROSBagToPandasDataFrame(_file)
        FINISHING_TIMES.append(FinishingTimeFirstRobotToCompleteMerkleTree(INITIAL_ID, num_of_bots, RUN_THRESHOLD, DF)[0])
        COMMUNICATION_GRAPHS.append(ExtractCommunicationGraphFirstRobotToCompleteMerkleTree(INITIAL_ID, num_of_bots, NUMBER_OF_LEAVES, DF))
        COMMUNICATION_COST.append(CommunicationCost(COMMUNICATION_GRAPHS[-1], NUMBER_OF_LEAVES, HASH_SIZE_IN_BYTES)/1024)
        PRIVACY_COEFFICIENT.append(PrivacyCoefficient(INITIAL_ID, num_of_bots, NUMBER_OF_LEAVES, DF))
        
        del DF
        gc.collect()

    COMPLETION_SUCCESS = CompletionSuccess(RUN_THRESHOLD, FINISHING_TIMES)      
    return FINISHING_TIMES, COMPLETION_SUCCESS, COMMUNICATION_GRAPHS, PRIVACY_COEFFICIENT, COMMUNICATION_COST

result = Parallel(n_jobs=-1)(delayed(RosbagDataProcessing)(i) for i in NUMBER_OF_ROBOTS_LIST)
FINISHING_TIMES, COMPLETION_SUCCESS, COMMUNICATION_GRAPHS, PRIVACY_COEFFICIENT, COMMUNICATION_COST = zip(*result)

# Single thread loop

In [ ]:
path=r'/home/eddie/experiments_results/real_robots/maze_formation'
ALL_FILES = glob2.glob(path + "/*/*/*.bag")
RUN_THRESHOLD = 5100
NUMBER_OF_ROBOTS_LIST = [16, 20, 24, 28]
INITIAL_ID = 20
NUMBER_OF_LEAVES = 16
HASH_SIZE_IN_BYTES = 32
LEAF_RANGE = range(16,17)

FINISHING_TIMES = [[] for i in range(len(NUMBER_OF_ROBOTS_LIST))]
COMPLETION_SUCCESS = [[] for i in range(len(NUMBER_OF_ROBOTS_LIST))]
COMMUNICATION_GRAPHS = [[] for i in range(len(NUMBER_OF_ROBOTS_LIST))]
PRIVACY_COEFFICIENT = [[] for i in range(len(NUMBER_OF_ROBOTS_LIST))]
COMMUNICATION_COST = [[] for i in range(len(NUMBER_OF_ROBOTS_LIST))]

for x, NUMBER_OF_ROBOTS in enumerate(NUMBER_OF_ROBOTS_LIST):
        regex = re.compile(r'R' + str(NUMBER_OF_ROBOTS))    
        FILTERED_FILES = filter(regex.search, ALL_FILES)
    
        for _file in FILTERED_FILES:
            DF = ROSBagToPandasDataFrame(_file)
            FINISHING_TIMES[x].append(FinishingTimeFirstRobotToCompleteMerkleTree(INITIAL_ID, NUMBER_OF_ROBOTS, RUN_THRESHOLD, DF)[0])
            COMMUNICATION_GRAPHS[x].append(ExtractCommunicationGraphFirstRobotToCompleteMerkleTree(INITIAL_ID, NUMBER_OF_ROBOTS, NUMBER_OF_LEAVES, DF))
            COMMUNICATION_COST[x].append(CommunicationCost(COMMUNICATION_GRAPHS[x][-1], NUMBER_OF_LEAVES, HASH_SIZE_IN_BYTES)/1024)
            PRIVACY_COEFFICIENT[x].append(PrivacyCoefficient(INITIAL_ID, NUMBER_OF_ROBOTS, NUMBER_OF_LEAVES, DF))

        COMPLETION_SUCCESS[x] = CompletionSuccess(RUN_THRESHOLD, FINISHING_TIMES)

# Write variables to pickle

In [18]:
with open('REALROBOT_MAZE_FINISHING_TIMES.pickle', 'wb') as f:
    pickle.dump(FINISHING_TIMES, f)
    
with open('REALROBOT_MAZE_COMMUNICATION_COST.pickle', 'wb') as f:
    pickle.dump(COMMUNICATION_COST, f)
            
with open('REALROBOT_MAZE_PRIVACY_COEEFICIENT.pickle', 'wb') as f:
    pickle.dump(PRIVACY_COEFFICIENT, f)    

# Read from pickle

In [27]:
with open('REALROBOT_MAZE_FINISHING_TIMES.pickle.pickle', 'rb') as f:
    FINISHING_TIMES = pickle.load(f)
with open('REALROBOT_MAZE_COMMUNICATION_COST.pickle', 'rb') as f:
    COMMUNICATION_COST = pickle.load(f)
with open('REALROBOT_MAZE_PRIVACY_COEEFICIENT.pickle', 'rb') as f:
    PRIVACY_COEFFICIENT= pickle.load(f)    

## Data storage

In [19]:
REALROBOT_MAZE_FINISHING_TIMES = FINISHING_TIMES
REALROBOT_MAZE_COMMUNICATION_COST = COMMUNICATION_COST
REALROBOT_MAZE_PRIVACY_COEEFICIENT = PRIVACY_COEFFICIENT

%store REALROBOT_MAZE_FINISHING_TIMES
%store REALROBOT_MAZE_COMMUNICATION_COST
%store REALROBOT_MAZE_PRIVACY_COEEFICIENT

Stored 'REALROBOT_MAZE_FINISHING_TIMES' (tuple)
Stored 'REALROBOT_MAZE_COMMUNICATION_COST' (tuple)
Stored 'REALROBOT_MAZE_PRIVACY_COEEFICIENT' (tuple)


In [29]:
%%time
df = pd.read_csv('/home/eddie/experiments_results/real_robots/R16/data/2020-05-22-20-01-41.bag.csv')

CPU times: user 4.17 s, sys: 240 ms, total: 4.41 s
Wall time: 4.41 s


In [12]:
def msg2json(msg):
   ''' Convert a ROS message to JSON format'''
   y = yaml.load(msg)
   return json.dumps(y,indent=4)

In [60]:
def position_message(ii):
    return json_message_converter.convert_json_to_ros_message('geometry_msgs/Pose', msg2json(ii)) 

def bool_message(ii):
    return json_message_converter.convert_json_to_ros_message('std_msgs/Bool', msg2json(ii)) 

def float_message(ii):
    return json_message_converter.convert_json_to_ros_message('std_msgs/Float32', msg2json(ii)) 

def string_message(ii):
    return json_message_converter.convert_json_to_ros_message('std_msgs/String', msg2json(ii)) 

def merkle_completed_message(ii):
    return json_message_converter.convert_json_to_ros_message('argos_ros_epuck/MerkleLeafList', msg2json(ii)) 
    
def neighbor_list_message(ii):
    return json_message_converter.convert_json_to_ros_message('argos_ros_epuck/NeighborList', msg2json(ii)) 
    
def proximity_message(ii):
    return json_message_converter.convert_json_to_ros_message('argos_ros_epuck/ProximityList', msg2json(ii)) 


positions=df[df['topic'].apply(lambda x: x[10:] == 'position')]
positions['msg'] = positions['msg'].map(position_message)

base_leds=df[df['topic'].apply(lambda x: x[10:] == 'base_leds')]
base_leds['msg'] = base_leds['msg'].map(bool_message)

battery=df[df['topic'].apply(lambda x: x[10:] == 'battery')]
battery['msg'] = battery['msg'].map(float_message)

# init_merkle_tree=df[df['topic'].apply(lambda x: x[10:] == 'comm/init_merkle_tree')]
# init_merkle_tree['msg'] = init_merkle_tree['msg'].map(string_message)

completed_merkle_tree=df[df['topic'].apply(lambda x: x[10:] == 'comm/init_merkle_tree')]
completed_merkle_tree['msg'] = completed_merkle_tree['msg'].map(merkle_completed_message)

neighbor_list=df[df['topic'].apply(lambda x: x[10:] == 'comm/neighbor_list')]
neighbor_list['msg'] = neighbor_list['msg'].map(neighbor_list_message)

proximities=df[df['topic'].apply(lambda x: x[10:] == 'proximity')]
proximities['msg'] = proximities['msg'].map(proximity_message)

#positions.head()

# def data(ii):
#     return json_message_converter.convert_json_to_ros_message('geometry_msgs/Pose', msg2json(ii)) 

# def neighbors(ii):
#     return json_message_converter.convert_json_to_ros_message('geometry_msgs/Pose', msg2json(ii)) 

final = pd.concat([positions, base_leds, battery, init_merkle_tree, completed_merkle_tree, neighbor_list, proximities])
final.sort_values(by=final.index, inplace=True)

# for i in df['msg'].values:
#     print(test_function(i))
#     msg = msg2json(i)
#     test = json_message_converter.convert_json_to_ros_message('geometry_msgs/Pose', msg)
#     print test.position
#     break

/home/eddie/anaconda3/envs/scipy/lib/python2.7/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/eddie/anaconda3/envs/scipy/lib/python2.7/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/eddie/anaconda3/envs/scipy/lib/python2.7/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pan

UnicodeDecodeError: 'utf8' codec can't decode byte 0xa2 in position 140: invalid start byte

In [17]:
%%time
MerkleTreeCompletionTimePerRobot(df, '/epuck_20/comm/merkle_completed')

1210

In [33]:
%%time

DF = ROSBagToPandasDataFrame('/home/eddie/2020-05-22-20-01-41.bag', convert_to_hdf5=True)

ImportError: HDFStore requires PyTables, "No module named tables" problem importing

In [ ]:
952 819 1186

In [28]:
%%time
MerkleTreeCompletionTimePerRobot(DF, '/epuck_20/comm/merkle_completed')

CPU times: user 73.6 ms, sys: 3 µs, total: 73.6 ms
Wall time: 72.2 ms


1210

In [54]:
FinishingTimeFirstRobotToCompleteMerkleTree(20, 16, 5100, df)

ValueError: min() arg is an empty sequence

In [ ]:
REALROBOT_FORAGING_COMMUNICATION_COST = COMMUNICATION_COST
